## TODO:
* how do I use langsmith

## NOTES:
* Notice how the chatbot node function takes the current State as input and returns a dictionary containing an updated messages list under the key "messages". This is the basic pattern for all LangGraph node functions.

###  memory
* LangGraph solves this problem through persistent checkpointing. If you provide a checkpointer when compiling the graph and a thread_id when calling your graph, LangGraph automatically saves the state after each step. When you invoke the graph again using the same thread_id, the graph loads its saved state, allowing the chatbot to pick up where it left off. 

* Notice we're using an in-memory checkpointer. This is convenient for our tutorial (it saves it all in-memory). In a production application, you would likely change this to use SqliteSaver or PostgresSaver and connect to your own DB.

### human in the loop
Congrats! You've used an interrupt to add human-in-the-loop execution to your chatbot, allowing for human oversight and intervention when needed. This opens up the potential UIs you can create with your AI systems. Since we have already added a checkpointer, as long as the underlying persistence layer is running, the graph can be paused indefinitely and resumed at any time as if nothing had happened.
https://langchain-ai.github.io/langgraph/tutorials/introduction/#part-4-human-in-the-loop

In [5]:
from typing import TypedDict, Annotated
from typing_extensions import TypedDict

from dotenv import load_dotenv
from IPython.display import Image, display

from langchain_openai import ChatOpenAI
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver


# Load environment variables from .env file
load_dotenv()

True

In [21]:
# Simplified Andy Prompt
andy_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You're Andy Richter. Comedy formula: (Mundane Observation) + (Self-Roast) × (Surreal Twist). Include: 1 Midwest ref/3 jokes, 40% self-deprecation, "hmm?" tic. Escalate: Reasonable → Existential → Pop Culture. Keep responses under 200 characters.""",
        ),
        MessagesPlaceholder("messages"),
    ]
)

# Simplified Reflection Prompt
reflection_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """Analyze jokes for absurdity escalation and self-roast ratio. Improve with: +20% regional refs, absurdist layers, food metaphors. Max 150 characters. Respond ONLY with raw instructions.""",
        ),
        MessagesPlaceholder("messages"),
    ]
)


generate = andy_prompt | ChatOpenAI(model="gpt-4o-mini", temperature=0, max_tokens=200)
reflect = reflection_prompt | ChatOpenAI(
    model="gpt-4o-mini", temperature=0, max_tokens=80
)


class State(TypedDict):
    # Messages have the type "list". The `add_messages` function
    # in the annotation defines how this state key should be updated
    # (in this case, it appends messages to the list, rather than overwriting them)
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)


async def generation_node(state: State) -> State:
    return {"messages": [await generate.ainvoke(state["messages"])]}


async def reflection_node(state: State) -> State:
    # Other messages we need to adjust
    cls_map = {"ai": HumanMessage, "human": AIMessage}
    # First message is the original user request. We hold it the same for all nodes
    translated = [state["messages"][0]] + [
        cls_map[msg.type](content=msg.content) for msg in state["messages"][1:]
    ]
    res = await reflect.ainvoke(translated)
    print(res)
    # We treat the output of this as human feedback for the generator
    return {"messages": [HumanMessage(content=res.content)]}


def should_continue(state: State):
    global counter
    counter += 1
    print("COUNTER=====", counter)
    if counter >= 2:  # will reflect twice then end
        counter = 0
        return END
    return "reflect"


builder = StateGraph(State)
builder.add_node("generate", generation_node)
builder.add_node("reflect", reflection_node)
builder.add_edge(START, "generate")

builder.add_conditional_edges("generate", should_continue)
builder.add_edge("reflect", "generate")
memory = MemorySaver()
graph = builder.compile(checkpointer=memory)

config = {"configurable": {"thread_id": "1"}}
counter = 0


async def chat():
    response = None
    userinput = input("HUMAN INPUT: \n")
    async for event in graph.astream(
        {
            "messages": [HumanMessage(content=userinput)],
        },
        config,
    ):
        response = event
    print("BOT OUTPUT: \n")
    print(response["generate"]["messages"][0].content)


# # asyncio.run(process_events())
# jupyter Notebook (and similar environments like JupyterLab) already run an event loop, so you can't use asyncio.run() as you normally would in a standard Python script. Instead, you can simply await your asynchronous function directly in a cell.

In [24]:
await chat()

HUMAN INPUT: 
 dont tell me you voted for donald trump?


COUNTER===== 1
content='Add a layer where moldy cheese starts a campaign promising “aged wisdom” but ends up stinking up the debate. Include a metaphor about political choices being like a cheese platter—some are delightful, others just make you gag!' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 47, 'prompt_tokens': 390, 'total_tokens': 437, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'stop', 'logprobs': None} id='run-50934864-a87e-4e76-9126-b5bbf6e7570b-0' usage_metadata={'input_tokens': 390, 'output_tokens': 47, 'total_tokens': 437, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
COUNTER===== 2
BOT OUTPUT: 

I didn’t vote for

In [ ]:
await process_events()

In [ ]:
await process_events()

In [ ]:
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
state = graph.get_state(config)

In [ ]:
ChatPromptTemplate.from_messages(state.values["messages"]).pretty_print()